### Proteomic data

In [1]:
import pandas as pd
df = pd.read_csv('/home/biodata/aman/data/data_raw/paxdb_human_data/9606-WHOLE_ORGANISM-integrated.txt', sep="\t")
df

,#gene_name,string_external_id,abundance
0,TMSB4X,9606.ENSP00000370010,8.296400e+04
1,APOA2,9606.ENSP00000356969,3.136100e+04
2,RBP4,9606.ENSP00000360522,2.444300e+04
3,ORM1,9606.ENSP00000259396,2.283500e+04
4,APOA1,9606.ENSP00000236850,2.072300e+04
...,...,...,...
19478,OR4N5,9606.ENSP00000493307,5.000000e-07
19479,hCG_1796489,9606.ENSP00000489727,4.000000e-07
19480,OR5M8,9606.ENSP00000323354,4.000000e-07
19481,C12orf42,9606.ENSP00000367353,3.000000e-07


# Should I do bootstrapping here? to get std and cv??

In [2]:
import geckopy
model = geckopy.io.read_sbml_ec_model('/home/biodata/aman/data/ecHumanGEM.xml', hardcoded_rev_reactions=False)  # how is this made actually? principle? # parameter for faster loading
print('Human Model is imported ... ') # 8+ mins # why ec model taking more time than the normal one?ODD

Human Model is imported ... 


In [3]:
# import pickle
# # with open("ecHumanGEM.pkl", "wb") as f:
# #     pickle.dump(model, f)
# model = pickle.load(open("/home/biodata/aman/data/ecHumanGEM.pkl", "rb")) ## to load
# import geckopy
# model = geckopy.io.read_sbml_ec_model(model)
# print('Human Model is imported ... ')

In [4]:
model.proteins

[<Protein prot_Q8IWW8[c] at 0x72df99937160>,
 <Protein prot_P28332[c] at 0x72df99937190>,
 <Protein prot_Q8N4Q0[c] at 0x72df99937fd0>,
 <Protein prot_P07327[c] at 0x72df99937f40>,
 <Protein prot_P40394[c] at 0x72df998c71f0>,
 <Protein prot_P00325[c] at 0x72df998c7280>,
 <Protein prot_P11766[c] at 0x72df998c7310>,
 <Protein prot_P08319[c] at 0x72df998c73a0>,
 <Protein prot_P00326[c] at 0x72df998c7430>,
 <Protein prot_P14550[c] at 0x72df998c74c0>,
 <Protein prot_Q9NR19[c] at 0x72df998c7550>,
 <Protein prot_Q9H6R3[c] at 0x72df998c7670>,
 <Protein prot_Q9NUB1[c] at 0x72df998c76a0>,
 <Protein prot_O00330[c] at 0x72df998c7070>,
 <Protein prot_P08559[c] at 0x72df998c77c0>,
 <Protein prot_P09622[c] at 0x72df998c7850>,
 <Protein prot_P10515[c] at 0x72df998c78e0>,
 <Protein prot_P11177[c] at 0x72df998c7970>,
 <Protein prot_P29803[c] at 0x72df998c7a00>,
 <Protein prot_P07195[c] at 0x72df998c7b20>,
 <Protein prot_P00338[c] at 0x72df998c7b50>,
 <Protein prot_Q8IX04[c] at 0x72df998c7730>,
 <Protein 

In [16]:
import requests, time, pandas as pd

df = pd.read_csv('/home/biodata/aman/data/data_raw/paxdb_human_data/9606-WHOLE_ORGANISM-integrated.txt', sep='\t', comment="#", header=None, names=["internal_id", "string_external_id", "abundance"], usecols = [0, 1, 2])
df

,internal_id,string_external_id,abundance
0,1849955,9606.ENSP00000317379,54.100
1,1853483,9606.ENSP00000349708,0.140
2,1853484,9606.ENSP00000349709,0.335
3,1853653,9606.ENSP00000350961,2.680
4,1848000,9606.ENSP00000298861,0.507
...,...,...,...
19944,1861401,9606.ENSP00000419160,0.496
19945,1846669,9606.ENSP00000280358,0.396
19946,1852835,9606.ENSP00000344566,0.909
19947,1843856,9606.ENSP00000247170,8.950


In [19]:
import requests, time, pandas as pd

df = pd.read_csv('/home/biodata/aman/data/data_raw/paxdb_human_data/9606-Adult_Lung_Kim_2014_SEQUEST.txt', sep='\t', comment="#", header=None, names=["internal_id", "string_external_id", "abundance"], usecols = [0, 1, 2])
# df = df[:19000] # testing

df["ENSP"] = df["string_external_id"].str.replace("9606.", "", regex=False) # this strips down to just the enseml tag

ensps = df["ENSP"].drop_duplicates().tolist()

all_results = []

batch = 25   # because UniProt seems to output only 25 items at a time # TODO: double-check

for i in range(0, len(ensps), batch):
    subset = ensps[i:i+batch]

    job = requests.post(
        "https://rest.uniprot.org/idmapping/run",
        data={
            "from": "Ensembl_Protein",
            "to": "UniProtKB",
            "ids": subset
        }
    ).json()

    job_id = job["jobId"]

    time.sleep(3)

    # Get results
    result = requests.get(
        f"https://rest.uniprot.org/idmapping/results/{job_id}"
    ).json()

    all_results.extend(result.get("results", []))

map_df = pd.DataFrame(all_results)
map_df = map_df.rename(columns={"from": "ENSP", "to": "UniProt"})

merged = df.merge(map_df, on="ENSP", how="left")

merged["protein_gecko_id"] = merged["UniProt"].apply(lambda x: f"prot_{x}[c]" if pd.notna(x) else None)

print(merged.head(20))
print("Total rows mapped:", merged["UniProt"].notna().sum())

    internal_id    string_external_id  abundance             ENSP UniProt  \
0       1853414  9606.ENSP00000349259   1467.000  ENSP00000349259  Q01082   
1       1848000  9606.ENSP00000298861      1.940  ENSP00000298861     NaN   
2       1853413  9606.ENSP00000349252      8.560  ENSP00000349252  P20701   
3       1849954  9606.ENSP00000317376     23.800  ENSP00000317376  P82912   
4       1849659  9606.ENSP00000313869      6.370  ENSP00000313869  Q9BQQ3   
5       1858827  9606.ENSP00000383333      0.017  ENSP00000383333  Q14149   
6       1850404  9606.ENSP00000321971     16.800  ENSP00000321971  Q8IXI1   
7       1862019  9606.ENSP00000436005     27.400  ENSP00000436005  Q9Y296   
8       1864209  9606.ENSP00000469669     22.100  ENSP00000469669  M0R1B8   
9       1844723  9606.ENSP00000259791    196.000  ENSP00000259791     NaN   
10      1842424  9606.ENSP00000205061     32.300  ENSP00000205061  Q92896   
11      1855795  9606.ENSP00000362063     18.300  ENSP00000362063  P33240   

In [20]:
prot = merged[["UniProt", "abundance"]].copy()
prot = prot.dropna(subset=["UniProt"])
prot = prot.rename(columns={"abundance": "copies_per_cell"})

prot["protein_gecko_id"] = prot["UniProt"].apply(lambda x: f"prot_{x}[c]")

prot["stdev"] = 0

len(prot)

6163

In [21]:
prot

,UniProt,copies_per_cell,protein_gecko_id,stdev
0,Q01082,1467.000,prot_Q01082[c],0
2,P20701,8.560,prot_P20701[c],0
3,P82912,23.800,prot_P82912[c],0
4,Q9BQQ3,6.370,prot_Q9BQQ3[c],0
5,Q14149,0.017,prot_Q14149[c],0
...,...,...,...,...
6661,P55196,26.500,prot_P55196[c],0
6662,Q6P988,3.760,prot_Q6P988[c],0
6663,Q12837,0.877,prot_Q12837[c],0
6664,P16989,67.100,prot_P16989[c],0


In [22]:
# prot

ec_model_exp = geckopy.experimental.from_copy_number(
    model,
    index=prot["protein_gecko_id"],
    cell_copies = prot["copies_per_cell"],
    stdev = prot["stdev"],
    vol=2.3e-12, dens=1.05, water=0.7
) # pipe output to additional log.f?

protein prot_Q01082[c] from measurements was not found in the model
protein prot_P20701[c] from measurements was not found in the model
protein prot_P82912[c] from measurements was not found in the model
protein prot_Q9BQQ3[c] from measurements was not found in the model
protein prot_Q14149[c] from measurements was not found in the model
protein prot_Q8IXI1[c] from measurements was not found in the model
protein prot_Q9Y296[c] from measurements was not found in the model
protein prot_M0R1B8[c] from measurements was not found in the model
protein prot_Q92896[c] from measurements was not found in the model
protein prot_P33240[c] from measurements was not found in the model
protein prot_O14776[c] from measurements was not found in the model
protein prot_Q8WVJ2[c] from measurements was not found in the model
protein prot_Q8N138[c] from measurements was not found in the model
protein prot_Q96CG8[c] from measurements was not found in the model
protein prot_A0A5K1VW95[c] from measurements was

In [23]:
model.proteins

[<Protein prot_Q8IWW8[c] at 0x72df99937160>,
 <Protein prot_P28332[c] at 0x72df99937190>,
 <Protein prot_Q8N4Q0[c] at 0x72df99937fd0>,
 <Protein prot_P07327[c] at 0x72df99937f40>,
 <Protein prot_P40394[c] at 0x72df998c71f0>,
 <Protein prot_P00325[c] at 0x72df998c7280>,
 <Protein prot_P11766[c] at 0x72df998c7310>,
 <Protein prot_P08319[c] at 0x72df998c73a0>,
 <Protein prot_P00326[c] at 0x72df998c7430>,
 <Protein prot_P14550[c] at 0x72df998c74c0>,
 <Protein prot_Q9NR19[c] at 0x72df998c7550>,
 <Protein prot_Q9H6R3[c] at 0x72df998c7670>,
 <Protein prot_Q9NUB1[c] at 0x72df998c76a0>,
 <Protein prot_O00330[c] at 0x72df998c7070>,
 <Protein prot_P08559[c] at 0x72df998c77c0>,
 <Protein prot_P09622[c] at 0x72df998c7850>,
 <Protein prot_P10515[c] at 0x72df998c78e0>,
 <Protein prot_P11177[c] at 0x72df998c7970>,
 <Protein prot_P29803[c] at 0x72df998c7a00>,
 <Protein prot_P07195[c] at 0x72df998c7b20>,
 <Protein prot_P00338[c] at 0x72df998c7b50>,
 <Protein prot_Q8IX04[c] at 0x72df998c7730>,
 <Protein 

In [24]:
ec_model_exp

Name,M_ecHumanGEM_v1__46__5__46__0
Memory address,72ccec624460
Number of metabolites,12220
Number of reactions,43084
Number of genes,3626
Number of groups,0
Objective expression,1.0*biomass_human - 1.0*biomass_human_reverse_fb2f2
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [25]:
ec_model_exp.slim_optimize()

7.494708447051393e-06